In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recomendation-pjct/movies.csv
/kaggle/input/movie-recomendation-pjct/ratings.csv


In [2]:
movie_df=pd.read_csv('/kaggle/input/movie-recomendation-pjct/movies.csv')
rating_df=pd.read_csv('/kaggle/input/movie-recomendation-pjct/ratings.csv')

In [3]:
movie_rating_df=movie_df.merge(rating_df,on='movieId')

In [4]:
movie_rating_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [5]:
# movie_rating_df.drop(columns=['timestamp'],inplace=True)
group=movie_rating_df.groupby('title')['rating'].agg(['mean','count']).sort_values(by='count',ascending=False).reset_index()

In [6]:
group

,title,mean,count
0,Forrest Gump (1994),4.164134,329
1,"Shawshank Redemption, The (1994)",4.429022,317
2,Pulp Fiction (1994),4.197068,307
3,"Silence of the Lambs, The (1991)",4.161290,279
4,"Matrix, The (1999)",4.192446,278
...,...,...,...
9714,King Solomon's Mines (1950),3.000000,1
9715,King Solomon's Mines (1937),2.500000,1
9716,King Ralph (1991),1.500000,1
9717,King Kong Lives (1986),2.000000,1


In [7]:
group.title.nunique()

9719

In [8]:
len(group.index)

9719

In [9]:
cutoff=group['count'].quantile(0.75)
print(cutoff)

9.0


In [10]:
group=group.loc[(group['mean']>=3) & (group['count']>=cutoff)]

In [11]:
len(group.index)

1981

In [12]:
movie_rating_df=movie_rating_df.loc[movie_rating_df.title.isin(group.title)]

In [13]:
movie_rating_df.title.nunique()

1981

In [14]:
movie_rating_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [15]:
group.head()

,title,mean,count
0,Forrest Gump (1994),4.164134,329
1,"Shawshank Redemption, The (1994)",4.429022,317
2,Pulp Fiction (1994),4.197068,307
3,"Silence of the Lambs, The (1991)",4.161290,279
4,"Matrix, The (1999)",4.192446,278


In [16]:
group['total']=group['mean']*group['count']

<ipython-input-16-245597e3562c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['total']=group['mean']*group['count']


In [17]:
group.head()

,title,mean,count,total
0,Forrest Gump (1994),4.164134,329,1370.0
1,"Shawshank Redemption, The (1994)",4.429022,317,1404.0
2,Pulp Fiction (1994),4.197068,307,1288.5
3,"Silence of the Lambs, The (1991)",4.161290,279,1161.0
4,"Matrix, The (1999)",4.192446,278,1165.5


In [18]:
import plotly.graph_objs as go 
trace=go.Bar(
    x=group['title'],
    y=group['total']
)
go.Figure(data=[trace],layout=go.Layout(title='title vs count'))

In [19]:
movie_rating_df.head()


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [20]:
pivot=movie_rating_df.pivot(index='userId',columns='movieId',values='rating')

In [21]:
pivot

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in greater



movieId,1,2,3,5,6,7,9,10,11,14,...,166643,168250,168252,171763,174055,175303,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,2.5,NaN,NaN,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
pivot.fillna(0,inplace=True)

In [23]:
pivot

movieId,1,2,3,5,6,7,9,10,11,14,...,166643,168250,168252,171763,174055,175303,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,2.5,0.0,0.0,2.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from surprise import SVD

In [25]:
svd=SVD()

In [26]:
movie_rating_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [27]:
input_df=movie_rating_df[['userId','movieId','rating']]

In [28]:
input_df.head()

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5


In [29]:
from surprise import Reader,Dataset,SVD

In [30]:
reader=Reader(rating_scale=(1,5))
training_data=Dataset.load_from_df(input_df[['userId','movieId','rating']],reader)

In [31]:
training_data=training_data.build_full_trainset()
svd=SVD()
svd.fit(training_data)

In [32]:
input_df.sample(10)

,userId,movieId,rating
79527,448,32587,4.0
54370,217,3448,3.0
98825,557,134130,3.5
6597,284,248,4.0
18076,19,724,4.0
35694,438,1784,4.0
82159,573,44665,4.5
78894,357,30793,3.0
45389,391,2580,4.0
92726,475,86332,5.0


In [33]:
print(svd.predict(uid=610,iid=1036))

user: 610        item: 1036       r_ui = None   est = 4.58   {'was_impossible': False}


In [34]:

movie_rating_df.loc[movie_rating_df['movieId']==1036,['title']].iloc[1,:]

title    Die Hard (1988)
Name: 22263, dtype: object

In [35]:
movie_rating_df.sample(10)

,movieId,title,genres,userId,rating,timestamp
92915,87222,Kung Fu Panda 2 (2011),Action|Adventure|Animation|Children|Comedy|IMAX,567,2.5,1525287842
25545,1204,Lawrence of Arabia (1962),Adventure|Drama|War,384,5.0,994038103
46361,2683,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy,135,4.0,1009692098
53237,3275,"Boondock Saints, The (2000)",Action|Crime|Drama|Thriller,91,4.5,1112713149
94599,95510,"Amazing Spider-Man, The (2012)",Action|Adventure|Sci-Fi|IMAX,534,3.5,1459787997
33005,1580,Men in Black (a.k.a. MIB) (1997),Action|Comedy|Sci-Fi,288,4.0,975691479
23108,1088,Dirty Dancing (1987),Drama|Musical|Romance,42,3.0,996259001
73749,7147,Big Fish (2003),Drama|Fantasy|Romance,51,4.0,1230931793
23512,1096,Sophie's Choice (1982),Drama,603,3.0,953928066
72714,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller,435,4.5,1366678158


In [36]:
pivot.head()

movieId,1,2,3,5,6,7,9,10,11,14,...,166643,168250,168252,171763,174055,175303,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
group.head()

,title,mean,count,total
0,Forrest Gump (1994),4.164134,329,1370.0
1,"Shawshank Redemption, The (1994)",4.429022,317,1404.0
2,Pulp Fiction (1994),4.197068,307,1288.5
3,"Silence of the Lambs, The (1991)",4.161290,279,1161.0
4,"Matrix, The (1999)",4.192446,278,1165.5


In [38]:
from sklearn.neighbors import NearestNeighbors

In [39]:
nn=NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(pivot)
distances,index=nn.kneighbors(pivot.iloc[3,:].values.reshape(1,-1),n_neighbors=7)

In [40]:
print(distances,index)

[[0.         0.61509682 0.63806471 0.63833744 0.65312277 0.66047169
  0.68193504]] [[  3 602 155 390 473 274 596]]
